In [2]:
import pandas as pd
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import sys
import numpy as np
# Make sure that your config.py Python file with your credentials is located
# in the same folder as your Jupyter notebook.
from config import *
import pickle

In [3]:
from sklearn.datasets import make_classification
from sklearn.cluster import HDBSCAN
from scipy.spatial.distance import euclidean
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from kneed import KneeLocator
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc
import time

In [4]:
from functions import*

In [5]:
from IPython.display import clear_output

In [6]:
from scipy.spatial import distance_matrix

In [7]:
from IPython.display import display, HTML

In [8]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias #
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=Client_ID,
                                                           client_secret=Client_Secret))

In [9]:
sp._session.timeout = 10

In [10]:
import pickle

def main():
    
    scaler_filename = "./scaler_v1.pickle"
    with open(scaler_filename, "rb") as file:
        scaler = pickle.load(file)
        
    umap_filename = "./umap.pickle"
    with open(umap_filename, "rb") as file:
        umap = pickle.load(file)
        
    database_filename = "./umap_df.csv"
    X_umap_df = pd.read_csv(database_filename)
    
    all_songs_filename = "../../Day_3/Afternoon/lab-clustering-songs/all_songs_clustered.csv"
    all_songs = pd.read_csv(all_songs_filename)
    all_songs['open_link'] = all_songs['id'].apply(lambda track_id: f'<a href="https://open.spotify.com/track/{track_id}" target="_blank">Click me</a>')
      
    answer = True
    while answer == True:
        # Ask the user to enter a song
        
        user_input = input("Please enter your song: ")
        
        five_songs = search_song_five(user_input)
        if five_songs.empty:
            print("Sorry, no songs found. Please enter a different song.")
            continue  # Ask for a new input if no songs are found
        five_songs_no_id=five_songs.drop(columns="id")
        display(five_songs_no_id)
        
        select_track = int(input("Please select your track from, giving a number between 0 and 4:"))
        song_id = five_songs.iloc[select_track,2]
        
        audio_feat=get_audio_features_for_chunks(sp, song_id,sleep_time=0)
        # How to proceed? Do we have to apply the whole df again 
        
        X = audio_feat.select_dtypes(np.number)
        
        #X_scaled = scaler.transform(X)
        #X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
        umap_representation = umap.transform(X)
        user_song_umap_df= pd.DataFrame(umap_representation,columns=["UMAP_1","UMAP_2"])
        d = distance_matrix(user_song_umap_df, X_umap_df.iloc[:,[0,1]]) #numpy array
        closest_song_to_user_song = np.argmin(d)
        cluster=X_umap_df.iloc[closest_song_to_user_song,-1]
        
        recommended_songs=get_records_by_id_and_cluster(all_songs,song_id,cluster)
        recommended_songs = recommended_songs[["artist","title","open_link"]]
        
        print("Please check the recommendations we have selected for you:\n\n")
        
        display(HTML(recommended_songs.to_html(escape=False)))

        # Ask the user if they want more recommendations
        user_response = input("Do you want to get more songs recommended? (yes/no): ").lower()
        while user_response not in ["yes", "no"]:
            print("\n")
            print("Please enter (yes/no)\n")
            user_response = input("Do you want to get more songs recommended? (yes/no): ").lower()
            print("\n")

        # Check if the user wants more recommendations
        if user_response == 'no':
            answer = False

In [11]:
main()

Please enter your song: greedy


,title,artist
0,greedy,Tate McRae
1,greedy,Tate McRae
2,greedy,Tate McRae
3,Greedy,Ariana Grande
4,Greedy,"King David, Tekno"


Please select your track from, giving a number between 0 and 4:0
Please check the recommendations we have selected for you:




,artist,title,open_link
21,jelly roll,need a favor,Click me
1,taylor swift,cruel summer,Click me
97,sophie ellis-bextor,murder on the dancefloor,Click me
50,drake,you broke my heart,Click me
4,zach bryan featuring kacey musgraves,i remember everything,Click me


Do you want to get more songs recommended? (yes/no): yes
Please enter your song: imagine


,title,artist
0,I Can Only Imagine,MercyMe
1,imagine,Ariana Grande
2,Imagine - Remastered 2010,John Lennon
3,Imagine,yvngxchris
4,Imagine,Jack Johnson


Please select your track from, giving a number between 0 and 4:2
Please check the recommendations we have selected for you:




,artist,title,open_link
1070,the ravens,gotta find my baby,Click me
196,alpha & omega,honest opinion,Click me
2112,brenda lee,fly me to the moon,Click me
2269,cinderella,night songs,Click me
3037,u-roy,peace and love,Click me


Do you want to get more songs recommended? (yes/no): rrr


Please enter (yes/no)

Do you want to get more songs recommended? (yes/no): sss




Please enter (yes/no)

Do you want to get more songs recommended? (yes/no): no




In [9]:
all_songs = pd.read_csv("../../Day_3/Afternoon/lab-clustering-songs/all_songs_clustered.csv")

In [10]:
all_songs.head()

,artist,title,id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,tag,hdbscan_cluster,dbscan_cluster
0,jack harlow,lovin on me,4xhsWYTOGcal8zt0J161CU,0.943,0.558,2,-4.911,1,0.0568,0.0026,...,104.983,audio_features,spotify:track:4xhsWYTOGcal8zt0J161CU,https://api.spotify.com/v1/tracks/4xhsWYTOGcal...,https://api.spotify.com/v1/audio-analysis/4xhs...,138411,4,hot,17,0
1,taylor swift,cruel summer,6ClYMObS7f3Nn4AiZHYQt3,0.278,0.907,9,-4.606,1,0.0827,0.1260,...,83.178,audio_features,spotify:track:6ClYMObS7f3Nn4AiZHYQt3,https://api.spotify.com/v1/tracks/6ClYMObS7f3N...,https://api.spotify.com/v1/audio-analysis/6ClY...,229567,4,hot,-1,1
2,tate mcrae,greedy,3rUGC1vUpkDG9CZFHMur1t,0.750,0.733,6,-3.180,0,0.0319,0.2560,...,111.018,audio_features,spotify:track:3rUGC1vUpkDG9CZFHMur1t,https://api.spotify.com/v1/tracks/3rUGC1vUpkDG...,https://api.spotify.com/v1/audio-analysis/3rUG...,131872,1,hot,123,2
3,doja cat,paint the town red,2IGMVunIBsBLtEQyoI1Mu7,0.868,0.538,5,-8.603,1,0.1740,0.2690,...,99.968,audio_features,spotify:track:2IGMVunIBsBLtEQyoI1Mu7,https://api.spotify.com/v1/tracks/2IGMVunIBsBL...,https://api.spotify.com/v1/audio-analysis/2IGM...,231750,4,hot,240,1
4,zach bryan featuring kacey musgraves,i remember everything,4KULAymBBJcPRpk1yO4dOG,0.429,0.453,0,-7.746,1,0.0459,0.5540,...,77.639,audio_features,spotify:track:4KULAymBBJcPRpk1yO4dOG,https://api.spotify.com/v1/tracks/4KULAymBBJcP...,https://api.spotify.com/v1/audio-analysis/4KUL...,227196,4,hot,259,1


In [11]:
all_songs['open_link'] = all_songs['id'].apply(lambda track_id: f'<a href="https://open.spotify.com/track/{track_id}" target="_blank">Click me</a>')

In [28]:
input_song = input("Please enter the name of song: ")

Please enter the name of song: greedy


In [ ]:
input_song

In [29]:
five_songs = search_song_five(input_song)

In [30]:
five_songs

,title,artist,id
0,greedy,Tate McRae,3rUGC1vUpkDG9CZFHMur1t
1,greedy,Tate McRae,4kfSXPK13aXkLzuz02hCSC
2,greedy,Tate McRae,2dHH3pG8xx0YOZDKyDD9eG
3,Greedy,Ariana Grande,1FFUXkoWIc9of8yOscGiaC
4,Greedy,"King David, Tekno",1FnIaiMNOQPwcWZJC7gw6V


In [31]:
row_selection = int(input("Please select the corresponding row: "))

Please select the corresponding row: 0


In [ ]:
row_selection

In [32]:
song_id = five_songs.iloc[row_selection,2]

In [33]:
song_id

'3rUGC1vUpkDG9CZFHMur1t'

In [34]:
audio_feat=get_audio_features_for_chunks(sp, song_id,sleep_time=0)

In [35]:
audio_feat

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.75,0.733,6,-3.18,0,0.0319,0.256,0,0.114,0.844,111.018,audio_features,3rUGC1vUpkDG9CZFHMur1t,spotify:track:3rUGC1vUpkDG9CZFHMur1t,https://api.spotify.com/v1/tracks/3rUGC1vUpkDG...,https://api.spotify.com/v1/audio-analysis/3rUG...,131872,1


In [36]:
X = audio_feat.select_dtypes(np.number)

In [37]:
X

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.75,0.733,6,-3.18,0,0.0319,0.256,0,0.114,0.844,111.018,131872,1


def scale_and_save(X, filename="scaler_v1.pickle"):
    """
    Scale the input data using StandardScaler, save the scaler object,
    and return the scaled data as a DataFrame.

    Parameters:
    - X: Input data (DataFrame or array-like)
    - filename: Path to save the scaler object (default: "scaler_v1.pickle")

    Returns:
    - X_scaled_df: Scaled data as a DataFrame
    """

    # Initialize StandardScaler
    scaler = StandardScaler()

    # Fit and transform the data
    X_scaled = scaler.fit_transform(X)

    # Save the scaler object to a file
    with open(filename, "wb") as file:
        pickle.dump(scaler, file)

    # Convert the scaled data to a DataFrame with column names
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

    return X_scaled_df

In [40]:
scaling_w_function=scale_and_save(X, "scaler_v1.pickle")

NameError: name 'StandardScaler' is not defined

In [ ]:
scaling_w_function

In [41]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
filename = "scaler_v1.pickle" # Path with filename

with open(filename, "wb") as file:
        pickle.dump(scaler,file)

X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
display(X.head())
print()
display(X_scaled_df.head())

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.75,0.733,6,-3.18,0,0.0319,0.256,0,0.114,0.844,111.018,131872,1


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_scaled

In [15]:
# We load the UMAP dimension reduction, which seems to be the one performing the best, as we saw on the previous lab

with open('umap.pickle', 'rb') as file:
    umap_model = pickle.load(file)

In [45]:
scaler = StandardScaler()

In [46]:
umap_representation = umap_model.transform(X)

In [47]:
umap_representation

array([[-1.9421761, 19.24299  ]], dtype=float32)

In [48]:
user_song_umap_df= pd.DataFrame(umap_representation,columns=["UMAP_1","UMAP_2"])

In [49]:
user_song_umap_df.head()

,UMAP_1,UMAP_2
0,-1.942176,19.24299


In [50]:
X_umap_transformed_df = pd.read_csv("umap_df.csv")

In [51]:
X_umap_transformed_df.head()

,UMAP_1,UMAP_2,cluster
0,1.329508,22.825453,-1
1,1.425039,22.725520,36
2,1.387273,22.768833,61
3,1.469725,22.683939,7
4,1.356548,22.794370,-1


In [52]:
X_umap_transformed_df_no_cluster = X_umap_transformed_df.drop(columns="cluster")

In [53]:
X_umap_transformed_df_no_cluster.head()

,UMAP_1,UMAP_2
0,1.329508,22.825453
1,1.425039,22.725520
2,1.387273,22.768833
3,1.469725,22.683939
4,1.356548,22.794370


In [54]:
d = distance_matrix(user_song_umap_df, X_umap_transformed_df_no_cluster) #numpy array
closest_song_to_user_song = np.argmin(d)

In [55]:
closest_song_to_user_song

1248

In [56]:
user_song_cluster = closest_song_to_user_song

In [57]:
user_song_cluster

1248

In [63]:
X_umap_transformed_df.loc[1248,"cluster"]

-1

In [61]:
all_songs.loc[598,"dbscan_cluster"]

53

In [62]:
song_cluster

NameError: name 'song_cluster' is not defined

def get_records_by_id_and_cluster(df, target_id, cluster_variable):
    # Check if the target_id is present in the ID column with the "hot" tag
    hot_records = df[(df['id'] == target_id) & (df['tag'] == 'hot')]

    if not hot_records.empty:
        # If "hot" records exist, return 5 records from the specific cluster
        hot_cluster_records = df[(df['tag'] == 'hot') & (df['hdbscan_cluster'] == cluster_variable)].sample(5)
        return hot_cluster_records
    else:
        # If "not_hot" records, return 5 records from the specific cluster
        not_hot_cluster_records = df[(df['tag'] == 'not_hot') & (df['hdbscan_cluster'] == cluster_variable)].sample(5)
        return not_hot_cluster_records

get_records_by_id_and_cluster(all_songs,song_id,song_cluster)

hot_records = all_songs[(all_songs['id'] == song_id) & (all_songs['tag'] == 'hot')]

if not hot_records.empty:
    # If "hot" records exist, return 5 records from the specific cluster
    hot_cluster_records = all_songs[(all_songs['tag'] == 'hot') & (all_songs['hdbscan_cluster'] == song_cluster)].sample(5)
    display(hot_cluster_records)
else:
    # If "not_hot" records, return 5 records from the specific cluster
    not_hot_cluster_records = all_songs[(all_songs['tag'] == 'not_hot') & (all_songs['hdbscan_cluster'] == song_cluster)].sample(5)
    display(not_hot_cluster_records)

In [ ]:
# Call the function to get audio features with a list of 1 id (user_song_id)
# Transform the user_song_audio_features
# Transform the features with UMAP
# Predict tghe user_song_cluster
# Determine if the user_song is hot (check is the user_song_id is amoung the hot_songs_ids)
# Idf[(full_df['set']=="Hot") & (full_df['hdbscan_cluster']==user_son_cluster)].sample(5)
# If not, full_df[(full_df['set']=="Not Hot") & (full_df['hdbscan_cluster']==user_son_cluster)].sample(5)
# ASk the user if he/she wants another recommendation, and update user_answerF so, full_

# https://open.spotify.com/track/id

In [65]:
main()

Please enter your song: greedy


,title,artist
0,greedy,Tate McRae
1,greedy,Tate McRae
2,greedy,Tate McRae
3,Greedy,Ariana Grande
4,Greedy,"King David, Tekno"


Please select your track from, giving a number between 0 and 4:0


ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
# Display the recommended songs
print("Recommended Songs:")
for song in recommended_songs:
    print(song)

In [ ]:
# Do he have to ransform the new all_songs clustered with UMAP?
# Do we have to scale the audio features for the user input and UMAP them? scaling gives 0 on every column. What is happening?
# If we have to umap both the main and the user df how to integrate them into the function
# how to integrate the scaler into the function?
